<table class="table table-bordered">
    <tr>
        <th style="text-align:center; vertical-align: middle; width:50%"><img src='https://www.np.edu.sg/images/default-source/default-album/img-logo.png'"></th>
        <th style="text-align:center;"><h1>Deep Learning in Natural Language Processing</h1><h2>Assignment (Problem 2) - Character Generator Model</h2><h3>AY2025/26 Semester</h3></th>
    </tr>
</table>

<h3>Name: Lim Ai Sim Elizabeth (3440680F)</h3>

### **Objective of this notebook**

1) Codes for model testing

- Loading required libraries
- functions for testing and evaluation
- loading the model and generating the text using the models for evaluation.

In [1]:
# FOR COLAB: Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# Import the Required Packages

from tensorflow import keras
print('keras: ', keras.__version__)

keras:  3.10.0


In [3]:
# FOR COLAB, copy the dataset to the temp data directory for training

import shutil
import os

# Define the source and destination paths
source_dir = '/content/drive/MyDrive/Colab Notebooks/03-NLP/assignment/Harry_Potter_Book.txt'
destination_dir = '/content/sample_data/Harry_Potter_Book.txt'  # Specify the destination file path

# Create the destination directory if it doesn't exist
os.makedirs(os.path.dirname(destination_dir), exist_ok=True)

# Copy the dataset
shutil.copy(source_dir, destination_dir)

print(f"Dataset copied from {source_dir} to {destination_dir}")

# update data directory
data_dir = '/content/sample_data/Harry_Potter_Book.txt'

Dataset copied from /content/drive/MyDrive/Colab Notebooks/03-NLP/assignment/Harry_Potter_Book.txt to /content/sample_data/Harry_Potter_Book.txt


In [4]:
import numpy as np

# path = keras.utils.get_file(
#     'Harry_Potter_Book.txt', origin='/')
# text = open('/content/sample_data/Harry_Potter_Book.txt').read().lower()
text = open('/content/sample_data/Harry_Potter_Book.txt').read() # leave capital letters be. last model 6-3
print('Corpus length:', len(text))
print(type(text))

Corpus length: 676978
<class 'str'>


In [5]:
import re

def remove_phrases(text, phrases):
    # Create a more flexible pattern that accounts for spaces and punctuation within words
    # For example, "j.k. rowling" can be matched as "j.k. *rowling", "j k rowling", etc.
    pattern = r'\b(?:' + '|'.join(re.escape(p).replace(r'\ ', r'\s*').replace(r'\.', r'\.*') for p in phrases) + r')\b'
    cleaned_text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    # Remove extra spaces after substitution
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

import re

def remove_phrases_strict(text, phrases):
    pattern_string = '|'.join(re.escape(p) for p in phrases)
    pattern = re.compile(pattern_string, flags=re.IGNORECASE)
    cleaned_text = pattern.sub('', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

phrases_to_remove_strict = [
    "Harry Potter and the Prisoner of Azkaban",
    "J.K.",
    "Rowling",
    "Page"
]

phrases_to_remove_flex = [
    "J.K.",
    "Rowling",
    ]


text = open('/content/sample_data/Harry_Potter_Book.txt').read()
text = text.replace('\n', ' ').replace('\r', ' ')
text = re.sub(r'[^a-zA-Z\s.,!?;:\"\'`]', '', text)

text = remove_phrases_strict(text, phrases_to_remove_strict)
text = remove_phrases(text, phrases_to_remove_flex)

print('Corpus length after strict cleaning:', len(text))
print(text[:500])

Corpus length after strict cleaning: 607752
OWL POST Harry Potter was a highly unusual boy in many ways. For one thing, he hated the summer holidays more than any other time of year. For another, he really wanted to do his homework but was forced to do it in secret, in the dead of night. And he also happened to be a wizard. It was nearly midnight, and he was lying on his stomach in bed, the blankets drawn right over his head like a tent, a flashlight in one hand and a large leatherbound book A History of Magic by Bathilda Bagshot propped 


In [6]:
# get some statistics on the text again

chars = sorted(list(set(text)))
print('Total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

print(indices_char)

# save indices_char dict
import pickle
with open('/content/sample_data/indices_char.pkl', 'wb') as f:
    pickle.dump(indices_char, f)

Total chars: 61
{0: ' ', 1: '!', 2: '"', 3: "'", 4: ',', 5: '.', 6: ':', 7: ';', 8: '?', 9: 'A', 10: 'B', 11: 'C', 12: 'D', 13: 'E', 14: 'F', 15: 'G', 16: 'H', 17: 'I', 18: 'J', 19: 'K', 20: 'L', 21: 'M', 22: 'N', 23: 'O', 24: 'P', 25: 'Q', 26: 'R', 27: 'S', 28: 'T', 29: 'U', 30: 'V', 31: 'W', 32: 'X', 33: 'Y', 34: 'Z', 35: 'a', 36: 'b', 37: 'c', 38: 'd', 39: 'e', 40: 'f', 41: 'g', 42: 'h', 43: 'i', 44: 'j', 45: 'k', 46: 'l', 47: 'm', 48: 'n', 49: 'o', 50: 'p', 51: 'q', 52: 'r', 53: 's', 54: 't', 55: 'u', 56: 'v', 57: 'w', 58: 'x', 59: 'y', 60: 'z'}


In [ ]:
# load pickle file

import pickle

with open('/content/sample_data/indices_char.pkl', 'rb') as f:
    indices_char = pickle.load(f)

In [ ]:
# Length of extracted character sequences
maxlen = 160

### Function for evaluating generated text

In [ ]:
# Repetitiveness and diversity

import re
from collections import Counter
import numpy as np

def evaluate_text_diversity(text):
    text = text.lower()
    words = re.findall(r'\b\w+\b', text)  # Extract words
    unique_words = set(words)
    unique_chars = set(text)

    # Repetition: how often is a word repeated consecutively?
    repeated_sequences = sum(1 for i in range(1, len(words)) if words[i] == words[i - 1])

    diversity_metrics = {
        'total_chars': len(text),
        'unique_char_count': len(unique_chars),
        'unique_word_count': len(unique_words),
        'unique_word_ratio': len(unique_words) / len(words) if words else 0,
        'repetition_count': repeated_sequences,
        'repetition_rate': repeated_sequences / len(words) if words else 0
    }

    return diversity_metrics


# Dictionary words coverage

import nltk
# Download the 'words' corpus if not already downloaded
try:
    nltk.data.find('corpora/words')
except nltk.downloader.DownloadError:
    nltk.download('words')
from nltk.corpus import words as nltk_words

english_vocab = set(w.lower() for w in nltk_words.words())

def evaluate_text_accuracy(text):
    text = text.lower()
    word_list = re.findall(r'\b\w+\b', text)

    known_words = [word for word in word_list if word in english_vocab]
    unknown_words = [word for word in word_list if word not in english_vocab]

    return {
        'total_words': len(word_list),
        'known_english_words': len(known_words),
        'unknown_words': len(unknown_words),
        'accuracy_ratio': len(known_words) / len(word_list) if word_list else 0
    }


def evaluate_generated_text(text):
    diversity = evaluate_text_diversity(text)
    accuracy = evaluate_text_accuracy(text)
    return {
        'diversity': diversity,
        'accuracy': accuracy
    }

# Define the sample function, assuming it's a helper for text generation
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


'''
Example use:


generated_text = "Harry Harry went to to the the castle castle of Hogwarts. It it was amazing."
result = evaluate_generated_text(generated_text)

from pprint import pprint
pprint(result)
'''

'\nExample use:\n\n\ngenerated_text = "Harry Harry went to to the the castle castle of Hogwarts. It it was amazing."\nresult = evaluate_generated_text(generated_text)\n\nfrom pprint import pprint\npprint(result)\n'

### Function for generating text

In [ ]:
# text generation for evaluation of trained model

import numpy as np

def generate_text(model, seed_text, length, chars, char_indices, maxlen, temperature=1.0):
    generated = seed_text
    for _ in range(length):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(generated[-maxlen:]): # Use the last 'maxlen' characters as input
            if char in char_indices:
                x_pred[0, t, char_indices[char]] = 1.0

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = chars[next_index]

        generated += next_char # Append the generated character

    return generated

### Testing Model 6-4

In [ ]:
# load the model

model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/03-NLP/assignment/PROBLEM2/p2_model6_4.keras')

In [ ]:
# takes the user input, print the number of characters submitted

text_input = input("Please enter a seed sentence: ")
print("Number of characters: ", len(text_input))

maxlen = 160


# pad text input up to maxlen or truncate the input to fit maxlen

# Check if text_input is shorter than maxlen and pad if necessary

if len(text_input) < maxlen:
    padded_text_input = text_input.rjust(maxlen, ' ') # Pad with spaces
    print("Padded text input:", padded_text_input)
elif len(text_input) > maxlen:
    padded_text_input = text_input[:maxlen] # Truncate if longer
    print("Truncated text input:", padded_text_input)
else:
    padded_text_input = text_input
    print("Text input:", padded_text_input)

Please enter a seed sentence: He watched a group of students in flowing black robes chatter excitedly as a majestic barn owl, a newly purchased pet, hooted softly from its cage. The street w
Number of characters:  160
Text input: He watched a group of students in flowing black robes chatter excitedly as a majestic barn owl, a newly purchased pet, hooted softly from its cage. The street w


In [ ]:
# encode the user input

x_pred = np.zeros((1, maxlen, len(chars)), dtype=bool)
for t, char in enumerate(padded_text_input):
    if char in char_indices:
        x_pred[0, t, char_indices[char]] = 1.0
    else:
        # Handle characters not in the training set, e.g., by ignoring them or using a placeholder
        print(f"Warning: Character '{char}' not found in vocabulary. Skipping.")

print("Shape of encoded input:", x_pred.shape)

Shape of encoded input: (1, 160, 61)


In [ ]:
# generate text in a range of temperatures

temps = [0.1, 0.3, 0.5, 0.7, 0.9]

for temp in temps:

  generated_text = generate_text(
    model=model,
    seed_text=padded_text_input,
    length=400,
    chars=chars,
    char_indices=char_indices,
    maxlen=maxlen,
    temperature=temp  # You can adjust the temperature here
  )

  evaluation_results = evaluate_generated_text(generated_text)

  print("Generated text:\n", generated_text)
  print("Temperature: ", temp)
  print("Evaluation Results:", evaluation_results)
  print("\n")



/tmp/ipython-input-2351409879.py:66: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


Generated text:
 He watched a group of students in flowing black robes chatter excitedly as a majestic barn owl, a newly purchased pet, hooted softly from its cage. The street with a staircan the staircancked the staircase of the stairs to the staircase to the front see the states and the train to the staircase of the stateres the train to the staircase to the train to the train to the face and the train to the states of the stairs to the staircase of the stairs to the staircase to the stairs to the stairs to the from the stairs to the train to the staircase to the train
Temperature:  0.1
Evaluation Results: {'diversity': {'total_chars': 560, 'unique_char_count': 26, 'unique_word_count': 39, 'unique_word_ratio': 0.3611111111111111, 'repetition_count': 0, 'repetition_rate': 0.0}, 'accuracy': {'total_words': 108, 'known_english_words': 93, 'unknown_words': 15, 'accuracy_ratio': 0.8611111111111112}}


Generated text:
 He watched a group of students in flowing black robes chatter excitedly

### Evaluation of Model 6-6

In [ ]:
# load the model

model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/03-NLP/assignment/PROBLEM2/p2_model6_6.keras')

In [ ]:
# takes the user input, print the number of characters submitted
text_input = input("Please enter a seed sentence: ")
print("Number of characters: ", len(text_input))

maxlen = 160

# pad text input up to maxlen or truncate the input to fit maxlen

# Check if text_input is shorter than maxlen and pad if necessary

if len(text_input) < maxlen:
    padded_text_input = text_input.rjust(maxlen, ' ') # Pad with spaces
    print("Padded text input:", padded_text_input)
elif len(text_input) > maxlen:
    padded_text_input = text_input[:maxlen] # Truncate if longer
    print("Truncated text input:", padded_text_input)
else:
    padded_text_input = text_input
    print("Text input:", padded_text_input)


Please enter a seed sentence: He watched a group of students in flowing black robes chatter excitedly as a majestic barn owl, a newly purchased pet, hooted softly from its cage. The street w
Number of characters:  160
Text input: He watched a group of students in flowing black robes chatter excitedly as a majestic barn owl, a newly purchased pet, hooted softly from its cage. The street w


In [ ]:
# encode the user input

x_pred = np.zeros((1, maxlen, len(chars)), dtype=bool)
for t, char in enumerate(padded_text_input):
    if char in char_indices:
        x_pred[0, t, char_indices[char]] = 1.0
    else:
        # Handle characters not in the training set, e.g., by ignoring them or using a placeholder
        print(f"Warning: Character '{char}' not found in vocabulary. Skipping.")

print("Shape of encoded input:", x_pred.shape)

Shape of encoded input: (1, 160, 61)


In [ ]:
# Use the loaded model to generate 400 characters with various temperature settings

temps = [0.1, 0.3, 0.5, 0.7, 0.9]

for temp in temps:

  generated_text = generate_text(
    model=model,
    seed_text=padded_text_input,
    length=400,
    chars=chars,
    char_indices=char_indices,
    maxlen=maxlen,
    temperature=temp
  )

  evaluation_results = evaluate_generated_text(generated_text)

  print("Generated text:\n", generated_text)
  print("Temperature: ", temp)
  print("Evaluation Results:", evaluation_results)
  print("\n")


/tmp/ipython-input-2351409879.py:66: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


Generated text:
 He watched a group of students in flowing black robes chatter excitedly as a majestic barn owl, a newly purchased pet, hooted softly from its cage. The street was the said the said to the sat the said to the someth and the sat the said the said to the start of the silend of the said to stopped and the spack to the said the sound and the said to the someth his tonight and the sound to the said to the said and the said and the saw to the said the said to the start and the sound of the sat the said and the stared to the silend and the start and the sound of
Temperature:  0.1
Evaluation Results: {'diversity': {'total_chars': 560, 'unique_char_count': 26, 'unique_word_count': 41, 'unique_word_ratio': 0.3445378151260504, 'repetition_count': 0, 'repetition_rate': 0.0}, 'accuracy': {'total_words': 119, 'known_english_words': 110, 'unknown_words': 9, 'accuracy_ratio': 0.9243697478991597}}


Generated text:
 He watched a group of students in flowing black robes chatter excitedly